In [4]:
import random
import numpy as np
import nltk
from nltk.corpus import wordnet

# Imports
from pyspark.sql.functions import col, lower, regexp_replace, when
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml import Pipeline

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer as KerasTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout

nltk.download('wordnet')
nltk.download('omw-1.4')

# 1. LOAD DATA 

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("IndicCorp Single Language Analysis") \
    .getOrCreate()

spark


csv_file_path = "data/train.csv"

print(f"📂 Loading data from: {csv_file_path}")

# 1. Read with header=False (Because your file has no header)
df = spark.read.csv(csv_file_path, header=False, inferSchema=True)

# 2. Rename the columns manually
# _c0 is Sentiment, _c1 is Title, _c2 is Review Text
df = df.withColumnRenamed("_c0", "sentiment") \
       .withColumnRenamed("_c1", "title") \
       .withColumnRenamed("_c2", "reviewText")

# 3. Create Label (Amazon data: 1=Negative, 2=Positive)
# We convert 2 -> 1 (Positive) and 1 -> 0 (Negative)
data = df.select("reviewText", "sentiment").dropna()
data = data.withColumn("label", (col("sentiment") == 2).cast("integer"))

print("✅ Data loaded and schema fixed!")

print("⚙️ Running Preprocessing Pipeline...")

# Clean punctuation
clean_data = data.withColumn("clean_text", regexp_replace(lower(col("reviewText")), "[^a-zA-Z\\s]", ""))

# Tokenize & Remove Stopwords
tokenizer = Tokenizer(inputCol="clean_text", outputCol="tokens")
remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")

pipeline = Pipeline(stages=[tokenizer, remover])
processed_df = pipeline.fit(clean_data).transform(clean_data)

# 3. SPLIT DATA (70/10/20)
print("✂️ Splitting Data...")
train_df, val_df, test_df = processed_df.randomSplit([0.7, 0.1, 0.2], seed=42)

train_pd = train_df.limit(70000).select("filtered_tokens", "label").toPandas()
val_pd = val_df.limit(10000).select("filtered_tokens", "label").toPandas()
test_pd = test_df.limit(30000).select("filtered_tokens", "label").toPandas()

# 4. PREPARE FOR BiLSTM (Keras)
train_texts = [" ".join(x) for x in train_pd['filtered_tokens']]
val_texts = [" ".join(x) for x in val_pd['filtered_tokens']]
test_texts = [" ".join(x) for x in test_pd['filtered_tokens']]

MAX_VOCAB = 20000
MAX_LEN = 100

k_tokenizer = KerasTokenizer(num_words=MAX_VOCAB)  #convert words into integers
k_tokenizer.fit_on_texts(train_texts)

X_train = pad_sequences(k_tokenizer.texts_to_sequences(train_texts), maxlen=MAX_LEN) # max length
X_val = pad_sequences(k_tokenizer.texts_to_sequences(val_texts), maxlen=MAX_LEN)
y_train = train_pd['label'].values
y_val = val_pd['label'].values

# 5. DEFINE BiLSTM MODEL
model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=128, input_length=MAX_LEN),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("\n🧠 Training BiLSTM Model...")
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=3, batch_size=64)

# 6. NOISE INJECTION
def remove_random_words(tokens, p=0.2):
    if not tokens: return []
    return [word for word in tokens if random.random() > p]

def introduce_spelling_mistakes(tokens, p=0.1):
    new_tokens = []
    for word in tokens:
        if len(word) > 3 and random.random() < p:
            idx = random.randint(0, len(word) - 2)
            word_list = list(word)
            word_list[idx], word_list[idx+1] = word_list[idx+1], word_list[idx] #swap 2 adjacent letters
            new_tokens.append("".join(word_list))
        else:
            new_tokens.append(word)
    return new_tokens

def synonym_replacement(tokens, p=0.2):
    new_tokens = []
    for word in tokens:
        if random.random() < p:
            synonyms = wordnet.synsets(word)
            if synonyms:
                lemma = synonyms[0].lemmas()[0].name()
                new_tokens.append(lemma.lower() if lemma.lower() != word.lower() else word)
            else:
                new_tokens.append(word)
        else:
            new_tokens.append(word)
    return new_tokens

# 7. EVALUATE ON NOISY DATA
print("\n🧪 Generating Noisy Test Set...")
noisy_test_texts = []

for tokens in test_pd['filtered_tokens']:
    t = remove_random_words(tokens, p=0.2)
    t = introduce_spelling_mistakes(t, p=0.1)
    t = synonym_replacement(t, p=0.2)
    noisy_test_texts.append(" ".join(t))

X_test_noisy = pad_sequences(k_tokenizer.texts_to_sequences(noisy_test_texts), maxlen=MAX_LEN)
y_test = test_pd['label'].values

loss, accuracy = model.evaluate(X_test_noisy, y_test)
print(f"\n✅ Final Accuracy on Noisy Data: {accuracy*100:.2f}%")

[nltk_data] Downloading package wordnet to /home/jaimin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jaimin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


📂 Loading data from: data/train.csv


✅ Data loaded and schema fixed!
⚙️ Running Preprocessing Pipeline...
✂️ Splitting Data...


/home/jaimin/miniconda3/envs/spark_env/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
E0000 00:00:1771384597.528635    6965 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1771384597.561044    6965 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...



🧠 Training BiLSTM Model...
Epoch 1/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 85s 74ms/step - accuracy: 0.7770 - loss: 0.4596 - val_accuracy: 0.8189 - val_loss: 0.3941
Epoch 2/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 79s 72ms/step - accuracy: 0.8560 - loss: 0.3290 - val_accuracy: 0.8214 - val_loss: 0.4053
Epoch 3/3
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 81s 74ms/step - accuracy: 0.8911 - loss: 0.2532 - val_accuracy: 0.8074 - val_loss: 0.4687

🧪 Generating Noisy Test Set...
938/938 ━━━━━━━━━━━━━━━━━━━━ 13s 14ms/step - accuracy: 0.7720 - loss: 0.5290

✅ Final Accuracy on Noisy Data: 77.20%


In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Sentiment").getOrCreate()

df = spark.read.csv("data/train.csv", header=False, inferSchema=True)

df = df.withColumnRenamed("_c0", "sentiment") \
       .withColumnRenamed("_c1", "title") \
       .withColumnRenamed("_c2", "reviewText")

data = df.select("reviewText", "sentiment").dropna()

print("Total rows:", data.count())
data.show(5, truncate=False)


26/02/18 09:24:59 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Total rows: 3599987
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|reviewText                                                                                                                                                                                                                                           

In [6]:
data = data.withColumn("label", (col("sentiment") == 2).cast("integer"))

data.groupBy("label").count().show()


+-----+-------+
|label|  count|
+-----+-------+
|    1|1799996|
|    0|1799991|
+-----+-------+



In [7]:
clean_data = data.withColumn(
    "clean_text",
    regexp_replace(lower(col("reviewText")), "[^a-zA-Z\\s]", "")
)

clean_data.select("reviewText", "clean_text").show(5, truncate=False)


+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
tokenizer = Tokenizer(inputCol="clean_text", outputCol="tokens")
tokenized_df = tokenizer.transform(clean_data)

tokenized_df.select("clean_text", "tokens").show(5, truncate=False)


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
from pyspark.sql.functions import size, avg

tokenized_df.select(avg(size(col("tokens")))).show()


+-----------------+
|avg(size(tokens))|
+-----------------+
|66.53845083329468|
+-----------------+



In [10]:
remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")
filtered_df = remover.transform(tokenized_df)

filtered_df.select("tokens", "filtered_tokens").show(5, truncate=False)


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
filtered_df.select(avg(size(col("filtered_tokens")))).show()


+--------------------------+
|avg(size(filtered_tokens))|
+--------------------------+
|         33.78088337541219|
+--------------------------+



In [12]:
from pyspark.sql.functions import explode

vocab_size = filtered_df.select(explode("filtered_tokens").alias("word")) \
                        .distinct() \
                        .count()

print("Vocabulary size:", vocab_size)


Vocabulary size: 1875203
